In [1]:
import geopandas as gpd
import rasterio
from shapely.geometry import Polygon, Point
from rasterio.mask import mask
import numpy as np
import pandas as pd

In [2]:
# Function to create grid
def create_grid(bbox, size):
    minx, miny, maxx, maxy = bbox
    x_coords = np.arange(minx, maxx, size)
    y_coords = np.arange(miny, maxy, size)
    polygons = []
    for x in x_coords:
        for y in y_coords:
            polygons.append(Polygon([
                (x, y), (x + size, y),
                (x + size, y + size), (x, y + size)
            ]))
    return gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")

In [ ]:
import rasterio
import numpy as np

# Open the raster file
raster_path = 'ESRI_LULC_2023_Rome.tif'

with rasterio.open(raster_path) as src:
    # Read the raster data as a NumPy array
    raster_data = src.read(1)  # Read the first (and typically only) band

    # Get the unique values
    unique_values = np.unique(raster_data)

# Print the unique values
print("Unique values in the raster:")
print(unique_values)


In [4]:
# Calculate LULC proportions
def calculate_lulc_proportions(raster_path, points, radius_in_degrees):
    with rasterio.open(raster_path) as src:
        results = []

        for point in points:
            buffer = point.buffer(radius_in_degrees)  # Convert meters to degrees
            buffered_geometry = [buffer]
            try:
                # Mask the raster to the buffer area
                out_image, out_transform = mask(src, buffered_geometry, crop=True)
                lulc_vals = out_image.flatten()
                # lulc_vals = lulc_vals[lulc_vals != 0]  # Remove nodata values

                # Total number of valid pixels in the buffer
                total_pixels = len(lulc_vals)

                # Initialize proportions dictionary
                proportions = {key: 0 for key in lulc_remap.values()}

                # Calculate proportion for each LULC code
                for key, name in lulc_remap.items():
                    code = int(key.split('_')[1])  # Extract numeric LULC code
                    code_count = np.sum(lulc_vals == code)  # Count pixels with this code
                    proportions[name] = code_count/total_pixels if total_pixels > 0 else 0

                results.append(proportions)
            except Exception as e:
                # Handle cases where the buffer goes out of bounds or errors occur
                results.append({key: 0 for key in lulc_remap.values()})
        return results


In [ ]:
# Bounding box of Rome
bounding_box = [12.24936, 41.71981, 12.63842, 42.08972]  # [min_lon, min_lat, max_lon, max_lat]

# Grid parameters
grid_size = 0.0008  # degrees

# 35 meters
buffer_radius = 0.0004 # degrees

# Define LULC remap dictionary
lulc_remap = {
    'lulc_0': 'no_data',
    'lulc_1': 'water',
    'lulc_2': 'trees',
    'lulc_4': 'flooded_veg',
    'lulc_5': 'crop',
    'lulc_7': 'built_area',
    'lulc_8': 'bare_ground',
    'lulc_9': 'snow_or_ice',
    'lulc_10': 'clouds',
    'lulc_11': 'range_land'
}


In [ ]:

# Create grid and centroids
grid = create_grid(bounding_box, grid_size)
grid['centroid'] = grid.geometry.centroid


In [ ]:
grid.head()

In [62]:

# Load LULC raster
lulc_raster = 'ESRI_LULC_2023_Rome.tif'
grid['lulc_proportions'] = calculate_lulc_proportions(lulc_raster, grid['centroid'], buffer_radius)

In [63]:
# Expand LULC proportions into separate columns
lulc_df = pd.DataFrame(grid['lulc_proportions'].tolist())
final_df = gpd.GeoDataFrame(pd.concat([grid.drop(columns=['lulc_proportions']), lulc_df], axis=1),
                            geometry='centroid', crs="EPSG:4326")

In [ ]:
final_df.head()

In [ ]:
final_df.tail()

In [67]:

# Optionally, save to a CSV file for further analysis
summary_stats.to_csv('summary_stats.csv')


In [ ]:


# Assuming `final_df` is your GeoDataFrame containing the LULC proportions

# Step 1: Set the centroid of each polygon as the geometry
final_df['geometry'] = final_df.centroid


In [73]:

# Step 2: Drop the original polygon column (geometry)
# Convert to a simple DataFrame with centroid geometry as point
final_df = final_df.drop(columns=['centroid'])


In [ ]:
final_df.head()

In [ ]:

# Step 3: Save the DataFrame as a CSV file
final_df.to_csv('lulc_proportions_centroids_40m.csv', index=False)

# Print the first few rows to check the output
print(final_df.head())

# Save to file or use further
final_df.to_file('rome_grid_2023_with_lulc_proportions_40m.gpkg', driver='GPKG')
